# Lab 2: Debug a Broken Vanilla GAN (find 12+ issues)

In [15]:
import torch, torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.1307,0.3081) # normalized data, so the mean = 0 and std = 1
    ])  # BUG normalize


loader=DataLoader(torchvision.datasets.MNIST('./data',True,download=True,transform=transform),batch_size=256,shuffle=True,num_workers=2,pin_memory=True)

z_dim=100; g_lr=0.00045; d_lr=0.00045 # previously the LR were too far apart (G: 2e-2, D: 2e-5) which is unstable - fix: changed the LR to be 0.00045 which makes the learning process more stable

class Disc(nn.Module):
    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(nn.Conv2d(1,32,4,2,1),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(32,64,4,2,1),
                               nn.BatchNorm2d(64),
                               nn.LeakyReLU(0.2,True),
                               nn.Conv2d(64,1,7,1,0),
                               ) # deleted the sigmoid activation function because it modifies the output before it's passed to the BCEwithlogits, which BCEwithlogits only deals with raw outputs

    def forward(self,x):return self.net(x).view(x.size(0),1)

class Gen(nn.Module):

    def __init__(self):
        super().__init__();
        self.net=nn.Sequential(nn.ConvTranspose2d(100,128,4,1,0),
                               nn.BatchNorm2d(128),
                               nn.ReLU(True),
                               nn.ConvTranspose2d(128,64,4,2,1),
                               nn.BatchNorm2d(64),
                               nn.ReLU(True),
                               nn.ConvTranspose2d(64,1,4,2,1),
                               nn.Tanh())  # BUG missing Tanh Solution: Added Tanh activation


    def forward(self,z):
      z=z.view(z.size(0),64,1,1);
      return self.net(z)  # BUG reshape


Dnet=Disc().to(device); Gnet=Gen().to(device)
crit=nn.BCEWithLogitsLoss()
opt_d=torch.optim.Adam(Dnet.parameters(),lr=d_lr,betas=(0.9,0.999))
opt_g=torch.optim.Adam(Gnet.parameters(),lr=g_lr,betas=(0.9,0.999))

In [16]:
from tqdm import tqdm
for real,_ in tqdm(loader):
    real=real.to(device);
    b=real.size(0)
    z=torch.randn(b,z_dim,device=device);
    loss_d=crit(Dnet(real),torch.zeros(b,1,device=device))+crit(Dnet(fake),torch.ones(b,1,device=device))  # BUG labels
    opt_d.zero_grad(); loss_d.backward(); opt_d.step()  # BUG is missing the zero grad and the step was opt_g when it should be opt_d. Fix: added the zero grad function for opt_d and corrected the step from opt_g.step to opt_d.step
    z=torch.randn(b,z_dim,device=device); fake=Gnet(z.view(b,z_dim,1,1))
    loss_g=crit(Dnet(fake),torch.ones(b,1,device=device))  # BUG non-sat problem: the loss_g took torch.zeros (it was inverted) meaning we're training the D to output 0 which is wrong. fix: i edited it from torch.zeros to torch.ones
    opt_g.zero_grad(); loss_g.backward(); opt_g.step() # BUG missing step. problem: missing zero grad and the opt_g step fix: Added opt_g.zero_grad() and opt_g.step()
print('Now fix all the issues.')

  0%|          | 0/235 [00:00<?, ?it/s]


NameError: name 'fake' is not defined